# Load DNS and mDNS datasets

In [1]:
import os
import sys

import numpy as np
import pandas as pd

module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
import torch_geometric.transforms as T

from libs.utils import score
from libs.loader import DNS

/home/local/QCRI/fdeniz/anaconda3/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Graphs

In [2]:
kg_path = lambda graph_name: f'./data/{graph_name}'

#### mDNS

In [3]:
# dataset = DNS(root=kg_path('mDNS'), transform=T.Compose([T.NormalizeFeatures(), T.ToUndirected()]), balance_gt=True)
# data = dataset[0]
# data['domain_node']['test_mask'].unique(return_counts=True)
# # datadomain_node.test_mask

#### DNS

In [4]:
dataset = DNS(root=kg_path('DNS'), transform=T.Compose([T.NormalizeFeatures(), T.ToUndirected()]), balance_gt=True)
data = dataset[0]
data['domain_node'].x.size()

Remove parallel edges: type
similar    50910
dtype: int64


torch.Size([373475, 10])

In [5]:
data = dataset.to_homogeneous()
print(data.train_mask.unique(return_counts=True))
print(data.test_mask.unique(return_counts=True))

(tensor([False,  True]), tensor([442682,   4386]))
(tensor([False,  True]), tensor([444437,   2631]))


In [6]:
from torch_geometric.nn import GATConv, HeteroConv, Linear


import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv, SAGEConv
import sys
sys.path.append('../')


class GNN(torch.nn.Module):
    def __init__(self, num_features, dim=16, num_classes=2, num_layers=2, model_type='gcn'):
        super(GNN, self).__init__()

        self.conv1 = SAGEConv(num_features, dim) if model_type == 'sage' else (GCNConv(num_features, dim) if model_type == 'gcn' else GATConv(num_features, dim))
        self.gcs = nn.ModuleList()
        self.num_layers = num_layers
        for i in range(1, num_layers):
            conv = SAGEConv(dim, dim) if model_type == 'sage' else (GCNConv(dim, dim) if model_type == 'gcn' else GATConv(dim, dim))
            self.gcs.append(conv)
        self.lin = Linear(dim, num_classes)

    def forward(self, x, edge_index, data=None, save_embedding=False):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        for i in range(1, self.num_layers):
            x = F.relu(self.gcs[i-1](x, edge_index))
            x = F.dropout(x, training=self.training)
        return self.lin(x)


    
model = GNN(data.x.size(1), dim=64, num_classes=2,
                  num_layers=2, model_type='gcn')

In [7]:
cuda_device = 3
torch.manual_seed(42)

if torch.cuda.is_available():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    torch.cuda.set_device(cuda_device)

    data, model = data.to(device), model.to(device)

with torch.no_grad():  # Initialize lazy modules.
    out = model(data.x, data.edge_index)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)


def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    mask = data.train_mask
    loss = F.cross_entropy(out[mask], data.y[mask])
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test():
    model.eval()
    pred = model(data.x, data.edge_index).argmax(dim=-1)

    accs = []
    for split in ['train_mask', 'val_mask', 'test_mask']:
        mask = data[split]
        acc = (pred[mask] == data.y[mask]).sum() / mask.sum()
        accs.append(float(acc))
    return accs


for epoch in range(1, 201):
    loss = train()
    train_acc, val_acc, test_acc = test()
    if epoch % 20 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, '
              f'Val: {val_acc:.4f}, Test: {test_acc:.4f}')
    
model.eval()
with torch.no_grad():
    pred = model(data.x, data.edge_index).argmax(dim=-1)
mask = data['test_mask']
scores = score(pred[mask],data.y[mask])
for metric, score in scores.items():
    print(metric, ':{:.2f}'.format(score))

Epoch: 020, Loss: 0.5904, Train: 0.6961, Val: 0.7071, Test: 0.6883
Epoch: 040, Loss: 0.5371, Train: 0.7462, Val: 0.7470, Test: 0.7507
Epoch: 060, Loss: 0.5047, Train: 0.7544, Val: 0.7470, Test: 0.7590
Epoch: 080, Loss: 0.4898, Train: 0.7654, Val: 0.7595, Test: 0.7765
Epoch: 100, Loss: 0.4880, Train: 0.7677, Val: 0.7550, Test: 0.7780
Epoch: 120, Loss: 0.4865, Train: 0.7638, Val: 0.7521, Test: 0.7735
Epoch: 140, Loss: 0.4836, Train: 0.7615, Val: 0.7550, Test: 0.7697
Epoch: 160, Loss: 0.4884, Train: 0.7656, Val: 0.7556, Test: 0.7739
Epoch: 180, Loss: 0.4890, Train: 0.7674, Val: 0.7590, Test: 0.7792
Epoch: 200, Loss: 0.4796, Train: 0.7699, Val: 0.7567, Test: 0.7826
tn, fp, fn, tp 1119 222 350 940
tn, fp, fn, tp 1119 222 350 940
acc :0.78
f1 :0.78
auc :0.78
prec :0.81
recall :0.73
fpr :0.17
